In [1]:
import gen

db = gen.Database("../.gen/default.db")

db.query("SELECT collection_name, sample_name, name FROM block_groups")

[['default', 'NULL', 'm123'],
 ['default', 'unknown', 'm123'],
 ['default', 'G1', 'm123'],
 ['default', 'foo', 'm123']]

In [5]:
# Directly from Rust
layout = gen.BaseLayout.from_graph(db, block_group_id=2)
layout.create_scaled_layout(label_width=200, scale=2)\
      .get_node_positions()

[((8, 1, 0, 0), ((1.0, 2.0), (1.0, 2.0))),
 ((1, 3, 0, 3), ((5.0, 2.0), (8.0, 2.0))),
 ((2, 3, 3, 5), ((12.0, 0.0), (14.0, 0.0))),
 ((0, 4, 0, 1), ((13.0, 4.0), (14.0, 4.0))),
 ((6, 8, 0, 4), ((26.0, 4.0), (30.0, 4.0))),
 ((5, 3, 11, 34), ((34.0, 2.0), (57.0, 2.0))),
 ((9, 2, 0, 0), ((61.0, 2.0), (61.0, 2.0))),
 ((4, 3, 9, 11), ((27.0, 0.0), (29.0, 0.0))),
 ((3, 3, 5, 9), ((18.0, 2.0), (22.0, 2.0)))]

In [4]:
# Through Python intermediate layer
layout = gen.GraphLayout("../.gen/default.db", 
                         block_group_id=2)
layout.create_scaled_layout(label_width=200, scale=2).get_node_positions()

[((5, 3, 11, 34), ((34.0, 2.0), (57.0, 2.0))),
 ((8, 1, 0, 0), ((1.0, 2.0), (1.0, 2.0))),
 ((4, 3, 9, 11), ((27.0, 0.0), (29.0, 0.0))),
 ((9, 2, 0, 0), ((61.0, 2.0), (61.0, 2.0))),
 ((1, 3, 0, 3), ((5.0, 2.0), (8.0, 2.0))),
 ((2, 3, 3, 5), ((12.0, 0.0), (14.0, 0.0))),
 ((6, 8, 0, 4), ((26.0, 4.0), (30.0, 4.0))),
 ((3, 3, 5, 9), ((18.0, 2.0), (22.0, 2.0))),
 ((0, 4, 0, 1), ((13.0, 4.0), (14.0, 4.0)))]